In [ ]:
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import numpy as np
import evaluate
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model, TaskType, PeftModel
import os

# Our training Data

In [12]:
data = [
    # Data for going home
    {"input": "I wanna get home", "output": "intent: navigate_home | filters: null |response: Okay, navigating you home."},
    {"input": "How do i get home?", "output": "intent: navigate_home | filters: null | response: Just follow the instractions on the map and you will arrive home."},
    {"input": "Where is my house?", "output": "intent: navigate_home | filters: null | response: Follow the navigation system to find your house."},
    {"input": "Take me home", "output": "intent: navigate_home | filters: null | response: Sure, let's go home."},
    
    # Data for Near by Health Facilities
    {"input": "Where are the closest hospitals?", "output": "intent: locate_hospitals | filters: hospitals | response: Here is a list of the closest hospitals."},
    {"input": "Are there any hospitals open now?", "output": "intent: locate_hospitals | filters: hospitals | response: These hospitals are currently open near you."},
    {"input": "Looking for a hospital around here.", "output": "intent: locate_hospitals | filters: hospitals | response: Got it. These hospitals are in your area."},

    {"input": "Can you show me a nearby clinic or pharmacy?", "output": "intent: locate_clinics | filters: both | response: Absolutely, here are clinics and pharmacies close by."},
    {"input": "Shown me the nearest health clinics.", "output": "intent: locate_clinics | filters: both | response: Sure thing! Those are the health facilities closest to you."},
    {"input": "What health facilities are around me?", "output": "intent: locate_clinics | filters: both | response: These are the health centers closest to your location."},

    {"input": "I need to find a pharmacy.", "output": "intent: locate_pharmacies | filters: pharmacies | response: No problem. Here are some in your area."},
    {"input": "Is there a 24-hour pharmacy around?", "output": "intent: locate_pharmacies | filters: pharmacies | response: Here's a list of 24-hour pharmacies nearby."},
    {"input": "I want a pharmacy.", "output": "intent: locate_pharmacies | filters: pharmacies | response: At your service. Want anything else?"},

    # SOS - Call Ambulence/ Help
    {"input": "Call for help", "output": "intent: navigate_sos | filters: fav_contacts | response: Okay, calling for help."},
    {"input": "Call an ambulance", "output": "intent: navigate_sos | filters: ambulance | response: Okay, calling an ambulance."},
    {"input": "Help, I need help", "output": "intent: navigate_sos | filters: fav_contacts | response: Okay, calling for help."},
    {"input": "Help, I don't feel good", "output": "intent: navigate_sos | filters: fav_contacts | response: Calling for help, sit tight."},
    {"input": "I need help", "output": "intent: navigate_sos | filters: fav_contacts | response: Okay, Calling for help."},

    # From here one they are the same data
    # Data for going home
    {"input": "I wanna get home", "output": "intent: navigate_home | filters: null |response: Okay, navigating you home."},
    {"input": "How do i get home?", "output": "intent: navigate_home | filters: null | response: Just follow the instractions on the map and you will arrive home."},
    {"input": "Where is my house?", "output": "intent: navigate_home | filters: null | response: Follow the navigation system to find your house."},
    {"input": "Take me home", "output": "intent: navigate_home | filters: null | response: Sure, let's go home."},
    
    # Data for Near by Health Facilities
    {"input": "Where are the closest hospitals?", "output": "intent: locate_hospitals | filters: hospitals | response: Here is a list of the closest hospitals."},
    {"input": "Are there any hospitals open now?", "output": "intent: locate_hospitals | filters: hospitals | response: These hospitals are currently open near you."},
    {"input": "Looking for a hospital around here.", "output": "intent: locate_hospitals | filters: hospitals | response: Got it. These hospitals are in your area."},

    {"input": "Can you show me a nearby clinic or pharmacy?", "output": "intent: locate_clinics | filters: both | response: Absolutely, here are clinics and pharmacies close by."},
    {"input": "Shown me the nearest health clinics.", "output": "intent: locate_clinics | filters: both | response: Sure thing! Those are the health facilities closest to you."},
    {"input": "What health facilities are around me?", "output": "intent: locate_clinics | filters: both | response: These are the health centers closest to your location."},

    {"input": "I need to find a pharmacy.", "output": "intent: locate_pharmacies | filters: pharmacies | response: No problem. Here are some in your area."},
    {"input": "Is there a 24-hour pharmacy around?", "output": "intent: locate_pharmacies | filters: pharmacies | response: Here's a list of 24-hour pharmacies nearby."},
    {"input": "I want a pharmacy.", "output": "intent: locate_pharmacies | filters: pharmacies | response: At your service. Want anything else?"},

    # SOS - Call Ambulence/ Help
    {"input": "Call for help", "output": "intent: navigate_sos | filters: fav_contacts | response: Okay, calling for help."},
    {"input": "Call an ambulance", "output": "intent: navigate_sos | filters: ambulance | response: Okay, calling an ambulance."},
    {"input": "Help, I need help", "output": "intent: navigate_sos | filters: fav_contacts | response: Okay, calling for help."},
    {"input": "Help, I don't feel good", "output": "intent: navigate_sos | filters: fav_contacts | response: Calling for help, sit tight."},
    {"input": "I need help", "output": "intent: navigate_sos | filters: fav_contacts | response: Okay, Calling for help."},
    # Data for going home
    {"input": "I wanna get home", "output": "intent: navigate_home | filters: null |response: Okay, navigating you home."},
    {"input": "How do i get home?", "output": "intent: navigate_home | filters: null | response: Just follow the instractions on the map and you will arrive home."},
    {"input": "Where is my house?", "output": "intent: navigate_home | filters: null | response: Follow the navigation system to find your house."},
    {"input": "Take me home", "output": "intent: navigate_home | filters: null | response: Sure, let's go home."},
    
    # Data for Near by Health Facilities
    {"input": "Where are the closest hospitals?", "output": "intent: locate_hospitals | filters: hospitals | response: Here is a list of the closest hospitals."},
    {"input": "Are there any hospitals open now?", "output": "intent: locate_hospitals | filters: hospitals | response: These hospitals are currently open near you."},
    {"input": "Looking for a hospital around here.", "output": "intent: locate_hospitals | filters: hospitals | response: Got it. These hospitals are in your area."},

    {"input": "Can you show me a nearby clinic or pharmacy?", "output": "intent: locate_clinics | filters: both | response: Absolutely, here are clinics and pharmacies close by."},
    {"input": "Shown me the nearest health clinics.", "output": "intent: locate_clinics | filters: both | response: Sure thing! Those are the health facilities closest to you."},
    {"input": "What health facilities are around me?", "output": "intent: locate_clinics | filters: both | response: These are the health centers closest to your location."},

    {"input": "I need to find a pharmacy.", "output": "intent: locate_pharmacies | filters: pharmacies | response: No problem. Here are some in your area."},
    {"input": "Is there a 24-hour pharmacy around?", "output": "intent: locate_pharmacies | filters: pharmacies | response: Here's a list of 24-hour pharmacies nearby."},
    {"input": "I want a pharmacy.", "output": "intent: locate_pharmacies | filters: pharmacies | response: At your service. Want anything else?"},

    # SOS - Call Ambulence/ Help
    {"input": "Call for help", "output": "intent: navigate_sos | filters: fav_contacts | response: Okay, calling for help."},
    {"input": "Call an ambulance", "output": "intent: navigate_sos | filters: ambulance | response: Okay, calling an ambulance."},
    {"input": "Help, I need help", "output": "intent: navigate_sos | filters: fav_contacts | response: Okay, calling for help."},
    {"input": "Help, I don't feel good", "output": "intent: navigate_sos | filters: fav_contacts | response: Calling for help, sit tight."},
    {"input": "I need help", "output": "intent: navigate_sos | filters: fav_contacts | response: Okay, Calling for help."},
    # Data for going home
    {"input": "I wanna get home", "output": "intent: navigate_home | filters: null |response: Okay, navigating you home."},
    {"input": "How do i get home?", "output": "intent: navigate_home | filters: null | response: Just follow the instractions on the map and you will arrive home."},
    {"input": "Where is my house?", "output": "intent: navigate_home | filters: null | response: Follow the navigation system to find your house."},
    {"input": "Take me home", "output": "intent: navigate_home | filters: null | response: Sure, let's go home."},
    
    # Data for Near by Health Facilities
    {"input": "Where are the closest hospitals?", "output": "intent: locate_hospitals | filters: hospitals | response: Here is a list of the closest hospitals."},
    {"input": "Are there any hospitals open now?", "output": "intent: locate_hospitals | filters: hospitals | response: These hospitals are currently open near you."},
    {"input": "Looking for a hospital around here.", "output": "intent: locate_hospitals | filters: hospitals | response: Got it. These hospitals are in your area."},

    {"input": "Can you show me a nearby clinic or pharmacy?", "output": "intent: locate_clinics | filters: both | response: Absolutely, here are clinics and pharmacies close by."},
    {"input": "Shown me the nearest health clinics.", "output": "intent: locate_clinics | filters: both | response: Sure thing! Those are the health facilities closest to you."},
    {"input": "What health facilities are around me?", "output": "intent: locate_clinics | filters: both | response: These are the health centers closest to your location."},

    {"input": "I need to find a pharmacy.", "output": "intent: locate_pharmacies | filters: pharmacies | response: No problem. Here are some in your area."},
    {"input": "Is there a 24-hour pharmacy around?", "output": "intent: locate_pharmacies | filters: pharmacies | response: Here's a list of 24-hour pharmacies nearby."},
    {"input": "I want a pharmacy.", "output": "intent: locate_pharmacies | filters: pharmacies | response: At your service. Want anything else?"},

    # SOS - Call Ambulence/ Help
    {"input": "Call for help", "output": "intent: navigate_sos | filters: fav_contacts | response: Okay, calling for help."},
    {"input": "Call an ambulance", "output": "intent: navigate_sos | filters: ambulance | response: Okay, calling an ambulance."},
    {"input": "Help, I need help", "output": "intent: navigate_sos | filters: fav_contacts | response: Okay, calling for help."},
    {"input": "Help, I don't feel good", "output": "intent: navigate_sos | filters: fav_contacts | response: Calling for help, sit tight."},
    {"input": "I need help", "output": "intent: navigate_sos | filters: fav_contacts | response: Okay, Calling for help."},
    # Data for going home
    {"input": "I wanna get home", "output": "intent: navigate_home | filters: null |response: Okay, navigating you home."},
    {"input": "How do i get home?", "output": "intent: navigate_home | filters: null | response: Just follow the instractions on the map and you will arrive home."},
    {"input": "Where is my house?", "output": "intent: navigate_home | filters: null | response: Follow the navigation system to find your house."},
    {"input": "Take me home", "output": "intent: navigate_home | filters: null | response: Sure, let's go home."},
    
    # Data for Near by Health Facilities
    {"input": "Where are the closest hospitals?", "output": "intent: locate_hospitals | filters: hospitals | response: Here is a list of the closest hospitals."},
    {"input": "Are there any hospitals open now?", "output": "intent: locate_hospitals | filters: hospitals | response: These hospitals are currently open near you."},
    {"input": "Looking for a hospital around here.", "output": "intent: locate_hospitals | filters: hospitals | response: Got it. These hospitals are in your area."},

    {"input": "Can you show me a nearby clinic or pharmacy?", "output": "intent: locate_clinics | filters: both | response: Absolutely, here are clinics and pharmacies close by."},
    {"input": "Shown me the nearest health clinics.", "output": "intent: locate_clinics | filters: both | response: Sure thing! Those are the health facilities closest to you."},
    {"input": "What health facilities are around me?", "output": "intent: locate_clinics | filters: both | response: These are the health centers closest to your location."},

    {"input": "I need to find a pharmacy.", "output": "intent: locate_pharmacies | filters: pharmacies | response: No problem. Here are some in your area."},
    {"input": "Is there a 24-hour pharmacy around?", "output": "intent: locate_pharmacies | filters: pharmacies | response: Here's a list of 24-hour pharmacies nearby."},
    {"input": "I want a pharmacy.", "output": "intent: locate_pharmacies | filters: pharmacies | response: At your service. Want anything else?"},

    # SOS - Call Ambulence/ Help
    {"input": "Call for help", "output": "intent: navigate_sos | filters: fav_contacts | response: Okay, calling for help."},
    {"input": "Call an ambulance", "output": "intent: navigate_sos | filters: ambulance | response: Okay, calling an ambulance."},
    {"input": "Help, I need help", "output": "intent: navigate_sos | filters: fav_contacts | response: Okay, calling for help."},
    {"input": "Help, I don't feel good", "output": "intent: navigate_sos | filters: fav_contacts | response: Calling for help, sit tight."},
    {"input": "I need help", "output": "intent: navigate_sos | filters: fav_contacts | response: Okay, Calling for help."}
]

# Flan-t5-base

In [13]:
# 2. Load Tokenizer and Model
model_name = "google/flan-t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# 3. Apply LoRA Configuration
# LoRA (Low-Rank Adaptation) is highly recommended for efficient fine-tuning,
# especially with smaller datasets, as it helps prevent catastrophic forgetting
# and reduces the number of trainable parameters, mitigating overfitting.
lora_config = LoraConfig(
    r=16, # Rank of the update matrices. Higher rank means more parameters, potentially better performance, but more prone to overfitting on small data.
    lora_alpha=32, # LoRA scaling factor
    target_modules=["q", "v"], # Common modules to apply LoRA to in T5 for performance
    lora_dropout=0.05, # Dropout for LoRA layers
    bias="none", # Don't train bias terms
    task_type=TaskType.SEQ_2_SEQ_LM, # Specify task type for T5
)
model = get_peft_model(model, lora_config)
model.print_trainable_parameters() # Shows how many parameters are actually trainable


# 4. Prepare Dataset for Training
# Convert your list of dictionaries to a Hugging Face Dataset
hf_dataset = Dataset.from_list(data)

def preprocess_function(examples):
    # Prepend 'summarize: ' or other task prefixes if your model requires it for FLAN-T5
    # FLAN-T5 is instruction-tuned, so you might not need a specific prefix for simple generation.
    # However, for consistency, you could add a generic instruction like:
    # "Please identify the intent and filters from the following query:"
    inputs = [f"Identify intent and filters: {item}" for item in examples["input"]]
    targets = examples["output"]

    model_inputs = tokenizer(inputs, max_length=128, truncation=True, padding="max_length")
    labels = tokenizer(targets, max_length=128, truncation=True, padding="max_length")

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_dataset = hf_dataset.map(preprocess_function, batched=True)

# 5. Training Arguments
output_dir = "./flan_t5_fine_tuned_model" # Directory to save checkpoints and final model
training_args = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=100,  # Start with a higher number; use early stopping
    per_device_train_batch_size=4, # Small batch size due to small data
    gradient_accumulation_steps=2, # Effectively larger batch size without more memory
    learning_rate=2e-4, # Fine-tuning learning rate
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    save_strategy="epoch", # Save checkpoint every epoch
    # eval_strategy="epoch", # Evaluate every epoch (requires a validation set)
    # load_best_model_at_end=True, # Load best model based on eval_metric
    metric_for_best_model="loss", # Metric to monitor for best model
    report_to="none", # Disable reporting to W&B or TensorBoard if not needed
    # Early stopping (crucial for small datasets)
    # If you had a separate validation set:
    # load_best_model_at_end=True,
    # save_total_limit=1, # Only save the best model
)

# 6. Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer,
    # data_collator=data_collator, # Not strictly necessary for simple Seq2Seq with padding="max_length"
)

# 7. Train the Model
print("Starting training...")
trainer.train()
print("Training complete.")


# 8. Save the Fine-Tuned Model
# When using PEFT, you typically save only the LoRA adapters.
# This saves disk space and allows you to easily switch between adapters.
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir) # Save the tokenizer as well
print(f"Model and tokenizer saved to {output_dir}")

trainable params: 1,769,472 || all params: 249,347,328 || trainable%: 0.7096


Map: 100%|██████████| 90/90 [00:00<00:00, 4154.38 examples/s]
C:\Users\vniko\AppData\Local\Temp\ipykernel_9076\2594627356.py:65: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForSeq2SeqLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Starting training...


Step,Training Loss
10,38.385900
20,29.518200
30,24.584100
40,20.173800
50,14.916100
60,9.019500
70,5.857000
80,4.755200
90,4.403100
100,4.154100


Training complete.
Model and tokenizer saved to ./flan_t5_fine_tuned_model


# Load and Testt model

In [ ]:
# To test the saved model locally
model_name = "google/flan-t5-base"
output_dir = "./flan_t5_fine_tuned_model" 
tokenizer = AutoTokenizer.from_pretrained(model_name)
loaded_model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
loaded_model = PeftModel.from_pretrained(loaded_model, output_dir)
loaded_model.eval()
text = "I need Help."
inputs = tokenizer(f"Identify intent and filters: {text}", return_tensors="pt")
outputs = loaded_model.generate(**inputs, max_new_tokens=50)
print(f"Test prediction: {tokenizer.decode(outputs[0], skip_special_tokens=True)}")

Test prediction: intent: navigate_sos | filters: fav_contacts | response: Okay, calling for help.
